In [1]:
#@title Installing needed dependencies!
from IPython.display import clear_output
try:
    from app_store_scraper import AppStore
    from pprint import pprint
    import pandas as pd
    import numpy as np
    import json
    print("Modules required are already installed!")
except ModuleNotFoundError:
    %pip install google-play-scraper
    clear_output()
    %pip install app-store-scraper
    clear_output()
    %pip install google-search-scraper-python
    clear_output()
    %pip install pandas
    clear_output()
    print("All modules required were sucessfully installed!")

Modules required are already installed!


In [ ]:
#@title App Reviews from Google Store
from google_play_scraper import app, search, Sort, reviews, reviews_all
from pprint import pprint
import pandas as pd
import numpy as np
import json
#static variables:
language = 'en'
countries = 'us'
appid = []

def searches(nameofSearch):
  result = search(
    nameofSearch,
    lang="en",  # defaults to 'en'
    country="us",  # defaults to 'us'
    n_hits=3  # defaults to 30 (= Google's maximum)
)
  return result

def getAppId(r):
  for d in r:
    id = (d['appId'])
    # appid.append(id)
    return id

def insertId(g, cont, rating, choice):

  if choice == None:
    if rating == None:
      result, continuation_token = reviews(
      g,
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.NEWEST
      count=cont # defaults to 100
      # defaults to None(means all score)
    )
      return result
    else:
      result, continuation_token = reviews(
      g,
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.NEWEST
      count=cont, # defaults to 100
      filter_score_with=rating# defaults to None(means all score)
    )
      return result
 
  else:
    if rating == None:
      result = reviews_all(
      g,
      sleep_milliseconds=0, # defaults to 0
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
      filter_score_with=None
      )
      return result
    else: 
      result = reviews_all(
      g,
      sleep_milliseconds=0, # defaults to 0
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      # sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
      filter_score_with=rating # defaults to None(means all score)
      )
      return result

def googleplay(a, ct, r, c):
  searches1 = searches(a)
  appId = getAppId(searches1)
  reviews = insertId(appId, ct, r, c)
  return reviews

def googleplay1(a, ct, r, c):
  searches1 = searches(a)
  appId = getAppId(searches1)
  reviews = insertId(appId, ct, r, c)
  return reviews 


appfile = open("googleplay-apps.txt", "r")
#appfile = open("C:\\Users\\nhant\\Research\\apps.txt", "r")
line = appfile.readline()
choice = "Yes" # All reviews
count = None
cond = "No" # No specific reviews
rating = None
while(line):
    app = line

    googleplaydf = pd.DataFrame(np.array(googleplay(app, count, rating, choice)),columns=['review'])
    googleplaydf2 = googleplaydf.join(pd.DataFrame(googleplaydf.pop('review').tolist()))
    #display(googleplaydf2)
    googleplaydf2.to_csv('googleplay-review-' + app.strip() + '.csv')

    line = appfile.readline()


In [ ]:
#@title App Reviews from App Store
from app_store_scraper import AppStore
from pprint import pprint
import pandas as pd
import numpy as np
import json

appfile = open("applestore-apps.txt", "r")
line = appfile.readline()
count = 100000

while(line):
    app = line.strip()

    applereviews = AppStore(country = "us", app_name = app)
    applereviews.review(how_many = count)
    applereviews.reviews
    applestoredf = pd.DataFrame(np.array(applereviews.reviews),columns=['review'])
    applestoredf2 = applestoredf.join(pd.DataFrame(applestoredf.pop('review').tolist()))
    
    applestoredf2.to_csv('applestore-review-' + app + '.csv')
    print(app)

    line = appfile.readline()

In [2]:
metadf = pd.DataFrame(columns = ['Name', 'ReviewCount', 'UniqueUserCount', 'AverageRating', 'AverageReviewLength', 'EarliestDate', 'LatestDate'])
appfile = open("common-apps.txt", "r")
line = appfile.readline()

while(line):
    app = line.strip()

    applestoredf = pd.read_csv('applestore-review-' + app + '.csv')
    googleplaydf = pd.read_csv('googleplay-review-' + app + '.csv')
    googleplaydf = googleplaydf.rename(columns={"content": "review", "score": "rating", "at": "date"})
    cdf = pd.concat([applestoredf, googleplaydf], ignore_index=True)

    new_row = pd.Series({'Name': app, 'ReviewCount': cdf.shape[0], 'UniqueUserCount': len(cdf.userName.value_counts()), 'AverageRating': cdf.rating.mean(), 'AverageReviewLength': cdf.review.str.len().mean(), 'EarliestDate': cdf.date.min(), 'LatestDate': cdf.date.max()})
    metadf = pd.concat([metadf, new_row.to_frame().T], ignore_index=True)
    print(app)

    line = appfile.readline()


codeacademy
datacamp
encode
learn-python-programiz
mimo
programming-hero
programming-hub
sololearn


In [3]:
print(metadf.to_markdown())

|    | Name                   |   ReviewCount |   UniqueUserCount |   AverageRating |   AverageReviewLength | EarliestDate        | LatestDate          |
|---:|:-----------------------|--------------:|------------------:|----------------:|----------------------:|:--------------------|:--------------------|
|  0 | codeacademy            |          3573 |              2911 |         3.75679 |               88.5665 | 2018-08-01 01:33:12 | 2023-03-24 22:07:12 |
|  1 | datacamp               |          7719 |              5087 |         4.47882 |               76.3687 | 2017-10-18 11:23:14 | 2023-03-24 21:09:12 |
|  2 | encode                 |          2045 |               450 |         4.63081 |               91.7667 | 2016-03-13 23:11:32 | 2023-03-19 01:47:34 |
|  3 | learn-python-programiz |          4814 |              4792 |         4.65351 |               60.8224 | 2019-08-22 19:39:09 | 2023-03-25 13:34:31 |
|  4 | mimo                   |         88724 |             52026 |         